In [2]:
!pip install lightgbm

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 1.2 MB 760 kB/s eta 0:00:01


In [1]:
import gc
import pandas as pd
from sklearn.model_selection import KFold,StratifiedKFold,train_test_split
from sklearn.metrics import f1_score,roc_auc_score,accuracy_score
import numpy as np
import lightgbm as lgb
import time

整合features

In [2]:
train_aggregate_df = pd.read_csv('output/features/train/aggregate_features_df.csv')
train_creative_id_embd_df = pd.read_csv('output/features/train/creative_id_embd_features_df.csv')
train_mode_df = pd.read_csv('output/features/train/mode_features_df.csv')

In [3]:
train_creative_id_embd_df

,user_id,age,gender,creative_id_embe_0_mean,creative_id_embe_0_max,creative_id_embe_0_min,creative_id_embe_1_mean,creative_id_embe_1_max,creative_id_embe_1_min,creative_id_embe_2_mean,...,creative_id_embe_28_min,creative_id_embe_29_mean,creative_id_embe_29_max,creative_id_embe_29_min,creative_id_embe_30_mean,creative_id_embe_30_max,creative_id_embe_30_min,creative_id_embe_31_mean,creative_id_embe_31_max,creative_id_embe_31_min
0,30920,2,1,2.696435,10.547447,-3.974081,1.842979,7.842483,-3.669027,0.331626,...,-2.893434,0.480825,5.808743,-6.121528,0.256153,6.621442,-8.093966,-2.843211,1.204376,-10.869002
1,309204,6,1,1.439296,6.164109,-0.033952,0.298971,3.209901,-1.278972,-0.118864,...,-1.712376,0.221683,3.663744,-1.817625,1.398582,4.682425,0.145295,-1.289117,-0.178416,-5.805322
2,309208,5,1,0.604122,4.568776,-5.144229,-0.881342,3.015404,-4.291122,-1.312271,...,-0.299478,2.132226,5.316924,0.163340,-0.691038,3.914967,-9.587008,-0.252022,1.097815,-2.857506
3,30921,3,1,1.469585,4.591361,-0.896838,-0.146451,3.251987,-2.208290,-0.121340,...,-4.996522,0.049099,5.007131,-3.343836,0.664738,4.874649,-1.223630,-0.731499,2.308530,-4.742735
4,309215,6,1,3.011093,15.938227,0.020577,1.740370,8.347221,-0.805572,-0.729477,...,-3.433952,-0.367808,2.613609,-5.337409,3.116013,8.522938,0.114052,-2.884347,1.141142,-8.022744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,30919,3,1,0.769057,8.684938,-8.312753,0.185966,6.620318,-5.128566,0.202453,...,-7.017665,0.509211,8.886160,-9.738061,0.787901,8.462729,-6.227891,-2.574125,3.180350,-12.676248
899996,309192,5,1,1.204220,8.323304,-3.569907,-0.453536,4.791988,-4.894526,-1.507068,...,-5.392109,0.628299,6.390893,-6.074021,-0.238192,3.700297,-8.237330,-0.377507,3.836577,-6.950735
899997,309194,6,1,2.459652,11.460368,-4.235135,0.767443,6.109372,-2.818472,0.241736,...,-2.915443,0.684601,3.207876,-3.096823,-0.802731,2.051512,-4.881048,-2.770491,0.414787,-8.526609
899998,309197,3,1,0.384850,6.671205,-5.112126,0.165058,4.640538,-4.527884,-0.264869,...,-4.001729,2.047363,10.463100,-1.437086,-0.119654,2.721082,-3.783841,-2.351107,0.021124,-7.254653


In [4]:
# #修正之前的保存失误
# del train_creative_id_embd_df['Unnamed: 0']
# del train_aggregate_df['Unnamed: 0']
# del train_mode_df['Unnamed: 0']
# gc.collect()
# train_aggregate_df.rename(columns={'user_id_': 'user_id'}, inplace=True)  

In [5]:
print('统计特征的维度：{0}'.format(train_aggregate_df.shape))
print('creative词向量的维度：{0}'.format(train_creative_id_embd_df.shape))
print('众数的维度：{0}'.format(train_mode_df.shape))

统计特征的维度：(900000, 13)
creative词向量的维度：(900000, 99)
众数的维度：(900000, 9)


In [6]:
train = pd.merge(train_mode_df, train_aggregate_df, on='user_id', how='left')
del train_mode_df, train_aggregate_df
gc.collect()
train = pd.merge(train, train_creative_id_embd_df, on='user_id', how='left')
del train_creative_id_embd_df
gc.collect()

0

In [7]:
train

,user_id,age_x,gender_x,creatice_id_mode,ad_id_mode,product_id_mode,product_category_mode,advertiser_id_mode,industry_mode,creative_id_nunique,...,creative_id_embe_28_min,creative_id_embe_29_mean,creative_id_embe_29_max,creative_id_embe_29_min,creative_id_embe_30_mean,creative_id_embe_30_max,creative_id_embe_30_min,creative_id_embe_31_mean,creative_id_embe_31_max,creative_id_embe_31_min
0,30920,2,1,2361397,2035987,\N,3,6783,6,37,...,-2.893434,0.480825,5.808743,-6.121528,0.256153,6.621442,-8.093966,-2.843211,1.204376,-10.869002
1,309204,6,1,819237,722657,26858,3,32066,242,24,...,-1.712376,0.221683,3.663744,-1.817625,1.398582,4.682425,0.145295,-1.289117,-0.178416,-5.805322
2,309208,5,1,37605,36260,\N,2,15855,5,9,...,-0.299478,2.132226,5.316924,0.163340,-0.691038,3.914967,-9.587008,-0.252022,1.097815,-2.857506
3,30921,3,1,5249,6473,\N,2,818,247,13,...,-4.996522,0.049099,5.007131,-3.343836,0.664738,4.874649,-1.223630,-0.731499,2.308530,-4.742735
4,309215,6,1,65824,60897,\N,3,24660,6,18,...,-3.433952,-0.367808,2.613609,-5.337409,3.116013,8.522938,0.114052,-2.884347,1.141142,-8.022744
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
899995,30919,3,1,1920050,1661869,\N,2,26007,6,80,...,-7.017665,0.509211,8.886160,-9.738061,0.787901,8.462729,-6.227891,-2.574125,3.180350,-12.676248
899996,309192,5,1,3580978,3083151,3789,13,30799,247,48,...,-5.392109,0.628299,6.390893,-6.074021,-0.238192,3.700297,-8.237330,-0.377507,3.836577,-6.950735
899997,309194,6,1,2049,2753,\N,2,6783,54,10,...,-2.915443,0.684601,3.207876,-3.096823,-0.802731,2.051512,-4.881048,-2.770491,0.414787,-8.526609
899998,309197,3,1,62032,57493,\N,18,10986,322,16,...,-4.001729,2.047363,10.463100,-1.437086,-0.119654,2.721082,-3.783841,-2.351107,0.021124,-7.254653


In [8]:
test_aggregate_df = pd.read_csv('output/features/test/aggregate_features_df.csv')
test_creative_id_embd_df = pd.read_csv('output/features/test/creative_id_embd_features_df.csv')
test_mode_df = pd.read_csv('output/features/test/mode_features_df.csv')

In [9]:
#修正之前的保存失误
del test_creative_id_embd_df['Unnamed: 0']
del test_aggregate_df['Unnamed: 0']
del test_mode_df['Unnamed: 0']
gc.collect()
test_aggregate_df.rename(columns={'user_id_': 'user_id'}, inplace=True)  

In [10]:
print('统计特征的维度：{0}'.format(test_aggregate_df.shape))
print('creative词向量的维度：{0}'.format(test_creative_id_embd_df.shape))
print('众数的维度：{0}'.format(test_mode_df.shape))

统计特征的维度：(1000000, 13)
creative词向量的维度：(1000000, 97)
众数的维度：(1000000, 7)


In [11]:
test = pd.merge(test_mode_df, test_aggregate_df, on='user_id', how='left')
del test_mode_df, test_aggregate_df
gc.collect()
test = pd.merge(test, test_creative_id_embd_df, on='user_id', how='left')
del test_creative_id_embd_df
gc.collect()

0

In [12]:
test

,user_id,creatice_id_mode,ad_id_mode,product_id_mode,product_category_mode,advertiser_id_mode,industry_mode,creative_id_nunique,click_times_max,click_times_min,...,creative_id_embe_28_min,creative_id_embe_29_mean,creative_id_embe_29_max,creative_id_embe_29_min,creative_id_embe_30_mean,creative_id_embe_30_max,creative_id_embe_30_min,creative_id_embe_31_mean,creative_id_embe_31_max,creative_id_embe_31_min
0,3131989,103304,93874,\N,2,37513,23,25,1,1,...,-1.539354,2.018382,9.637029,-0.043096,-0.824158,1.679926,-5.602281,1.779867,4.088388,-0.218126
1,3131990,1232695,1078353,\N,18,10989,6,37,3,1,...,-8.324322,1.198425,8.510142,-5.374815,-1.313662,4.547346,-8.137257,-1.400885,1.240196,-6.777824
2,3131994,143002,128468,23,2,2536,247,13,2,1,...,-2.937590,0.865044,3.067786,-0.171666,-0.989933,0.096906,-5.509692,-0.536061,-0.090365,-2.286924
3,3131995,32571,31960,1377,2,22123,5,14,2,1,...,-1.490991,-1.220434,3.417681,-7.556056,0.983572,4.958873,-2.659328,-1.612444,2.192160,-7.265713
4,3132000,1443125,1257879,\N,2,10334,6,17,2,1,...,-7.267167,-1.738678,4.896663,-5.915945,0.762109,6.712760,-5.990463,-1.530350,3.589237,-7.580082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,3131979,760162,671936,\N,18,14682,6,33,2,1,...,-4.497108,0.458831,9.418974,-2.767749,0.392901,7.470737,-6.244327,0.140590,5.350230,-3.235223
999996,3131983,3507883,3021200,\N,18,14681,6,9,1,1,...,-0.670872,-0.312979,2.795393,-2.442508,-0.286127,2.167267,-4.085524,-2.856222,-0.066788,-6.281056
999997,3131984,214243,192622,1634,2,16636,\N,23,2,1,...,-2.382609,-0.706722,2.209116,-4.764629,-0.293348,7.250177,-8.372250,-0.905386,1.610920,-6.618173
999998,3131987,74072,68290,\N,18,14681,6,15,1,1,...,-2.841492,-1.283381,1.771665,-6.171351,0.991014,5.360950,-0.393036,1.702859,4.753871,-0.483625


In [17]:
features = [ x for x in train.columns if x not in ['user_id', 'age', 'gender']]

In [18]:
len(features)

112

In [19]:
# #去前1000做demo
# train = train[0:1000]
# test = test[0:1000]

In [20]:
target1 = train['age'] - 1
target2 = train['gender'] - 1

KeyError: 'age'

In [17]:
params2 = {
    'objective': 'binary',
        'metric': 'auc',
        'learning_rate': 0.11
}

In [18]:
params1 = {
#     'max_depth':14,
#     'num_leaves': 64,
#     'min_data_in_leaf': 20,
#     'min_child_samples':20,
    'objective': 'multiclassova',
    'num_class': 10,
    'learning_rate': 0.1,
    "boosting": "gbdt",
#         'max_bin': 255,
#     'bagging_fraction': 0.85,
#     "feature_fraction": 0.8,
#     "bagging_freq": 1,
#     "bagging_fraction": 0.85,
#     "bagging_seed": 23,
#     "metric": 'logloss',
#     "lambda_l1": 0.2,
#     "nthread": 4,
}

In [18]:
def lgb_model_2(n_splits, random_state, train, test, features, target, params):
    start = time.clock()
    folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    #验证集
    oof_lgb = np.zeros(len(train))
    #测试集
    predictions_lgb = np.zeros(len(test))

    for fold_, (train_idx, valid_idx) in enumerate(folds.split(train[features].values, target.values)):
        print('fold：{} start...'.format(fold_))
        #训练集和验证集的划分
        train_data = lgb.Dataset(train[features].iloc[train_idx], label=target.iloc[train_idx])
        valid_data = lgb.Dataset(train[features].iloc[valid_idx], label=target.iloc[valid_idx])
        
        clf = lgb.train(
            params,
            train_data,
            valid_sets = [train_data, valid_data],
            verbose_eval = 1000,
            early_stopping_rounds=100 
        )
        
        del train_data, valid_data
        gc.collect()
        
        #对验证集进行预测
        val_pred = clf.predict(train[features].iloc[valid_idx], num_iteration=clf.best_iteration) # 每折 验证集的概率
        oof_lgb[valid_idx] = val_pred                    #整个  验证集的概率
        val_target = target2.iloc[valid_idx]             #验证集的真实标签
        val_pred =(val_pred>=0.499).astype('int')        #验证集的预测标签
        print("fold_{}：valid_acc：{}".format(fold_, accuracy_score(val_target, val_pred)))   #每折  验证集的ACC
        
        #对测试集进行预测
        predictions_lgb += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits
        
        print('fold：{} end...\n'.format(fold_))
    
    predictions_lgb = (predictions_lgb>=0.499).astype('int')
    
    print('评分信息')
    print("ROC: {:<8.5f}".format(roc_auc_score(target2, oof_lgb)))
    valid_pre = (oof_lgb>=0.499).astype('int')
    gender_acc = accuracy_score(target2,valid_pre)
    print("accuracy: {:<8.5f}".format(gender_acc))
    
    print("run time：{}".format(time.clock() - start))
    return valid_pre, predictions_lgb

In [19]:
def lgb_model_10(n_splits, random_state, train, test, features, target, params):
    start = time.clock()
    folds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    
    #验证集
    oof_lgb = np.zeros((len(train), 10))
    #测试集
    predictions_lgb = np.zeros((len(test), 10))

    for fold_, (train_idx, valid_idx) in enumerate(folds.split(train[features].values, target.values)):
        print('fold：{} start...'.format(fold_))
        #训练集和验证集的划分
        train_data = lgb.Dataset(train[features].iloc[train_idx], label=target.iloc[train_idx])
        valid_data = lgb.Dataset(train[features].iloc[valid_idx], label=target.iloc[valid_idx])
        
        clf = lgb.train(
            params,
            train_data,
            valid_sets = [train_data, valid_data],
            verbose_eval = 1000,
            early_stopping_rounds=100 
        )
        
        del train_data, valid_data
        gc.collect()
        
        #对验证集进行预测
        val_pred = clf.predict(train[features].iloc[valid_idx], num_iteration=clf.best_iteration)
        oof_lgb[valid_idx] = val_pred
        val_target = target2.iloc[valid_idx]
        val_pred = np.argmax(val_pred, axis=1)
        print("fold_{}：valid_F1：{}".format(fold_, f1_score(val_target, val_pred, average='macro')))
        
        #对测试集进行预测
        predictions_lgb += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits
        
        print('fold：{} end...\n'.format(fold_))
    
    predictions_lgb = np.argmax(predictions_lgb, axis=1)
    
    print('评分信息')
    valid_pre = np.argmax(oof_lgb, axis=1)    
    print("F1:",f1_score(valid_pre,target,average='macro'))
    print("run time：{}".format(time.clock() - start))

    return valid_pre, predictions_lgb

In [22]:
valid_pre_gender, pre_gender = lgb_model_2(5, 2020, train, test, features, target2, params2)

fold：0 start...
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.959517	valid_1's auc: 0.958525
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.959517	valid_1's auc: 0.958525
fold_0：valid_acc：0.9055555555555556
fold：0 end...

fold：1 start...
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.959709	valid_1's auc: 0.957112
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.959709	valid_1's auc: 0.957112
fold_1：valid_acc：0.9045722222222222
fold：1 end...

fold：2 start...
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.959631	valid_1's auc: 0.9581
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.959631	valid_1's auc: 0.9581
fold_2：valid_acc：0.9058055555555555
fold：2 end...

fold：3 start...
Training until validation scores don't improve for 100 rounds
[100]	training's auc: 0.959826	valid_1's auc: 0.957387
Did not me

In [29]:
test['pre_gender'] = pre_gender

In [37]:
sub = test[['user_id', 'pre_gender']]
sub['predicted_gender'] = sub['pre_gender'] + 1

/opt/conda/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [38]:
sub['predicted_age'] = -1

/opt/conda/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [40]:
sub = sub[['user_id', 'predicted_gender', 'predicted_age']]

In [41]:
sub

,user_id,predicted_gender,predicted_age
0,3131989,1,-1
1,3131990,1,-1
2,3131994,1,-1
3,3131995,1,-1
4,3132000,1,-1
...,...,...,...
999995,3131979,1,-1
999996,3131983,1,-1
999997,3131984,2,-1
999998,3131987,1,-1


In [42]:
sub.to_csv("result_gender.csv",index=False,sep=',')

In [20]:
valid_pre_age, pre_age = lgb_model_10(5, 2020, train, test, features, target1, params1)

fold：0 start...
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	training's multi_logloss: 1.59804	valid_1's multi_logloss: 1.64899
fold_0：valid_F1：0.02343304029959877
fold：0 end...

fold：1 start...
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	training's multi_logloss: 1.59756	valid_1's multi_logloss: 1.65028
fold_1：valid_F1：0.02325539330823439
fold：1 end...

fold：2 start...
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	training's multi_logloss: 1.59722	valid_1's multi_logloss: 1.65093
fold_2：valid_F1：0.023514654569501404
fold：2 end...

fold：3 start...
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[100]	training's multi_logloss: 1.59798	valid_1's multi_logloss: 1.65053
fold_3：valid_F1：0.023435838556757006
fold：3 end...

In [21]:
test['predicted_age'] = pre_age + 1

In [22]:
sub = test[['user_id', 'predicted_age']]

In [23]:
sub['predicted_age'] = sub['predicted_age'] + 1

/opt/conda/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [24]:
sub['predicted_gender'] = -1

/opt/conda/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [25]:
sub

,user_id,predicted_age,predicted_gender
0,3131989,2,-1
1,3131990,2,-1
2,3131994,2,-1
3,3131995,6,-1
4,3132000,6,-1
...,...,...,...
999995,3131979,10,-1
999996,3131983,9,-1
999997,3131984,5,-1
999998,3131987,5,-1


In [26]:
sub.to_csv("result_age.csv",index=False,sep=',')